In [8]:
import json
import pathlib
import re
import os

import pandas as pd
import numpy as np


In [9]:
def open_result_ipynb(path: pathlib.Path):
    with open(path, 'r') as f:
        return json.loads(''.join(f.readlines()))

In [10]:
def parse_result_dataframe(mean_list,
                           std_list,
                           alg,
                           exp,
                           dataset) -> pd.DataFrame:
    mean_precision = float(re.sub('\s+', ' ', mean_list[0].strip()).split()[1])
    mean_recall = float(re.sub('\s+', ' ', mean_list[1].strip()).split()[1])
    mean_f1_score = float(re.sub('\s+', ' ', mean_list[2].strip()).split()[1])
    std_precision = float(re.sub('\s+', ' ', std_list[0].strip()).split()[1])
    std_recall = float(re.sub('\s+', ' ', std_list[1].strip()).split()[1])
    std_f1_score = float(re.sub('\s+', ' ', std_list[2].strip()).split()[1])

    return pd.DataFrame({
        'dataset': [dataset],
        'algorithm': [alg],
        'experiment': [exp],
        'mean_precision': [mean_precision],
        'std_precision': [std_precision],
        'mean_recall': [mean_recall],
        'std_recall': [std_recall],
        'mean_f1_score': [mean_f1_score],
        'std_f1_score': [std_f1_score]
    })


In [11]:
def results_for(df, dataset, alg):

    # Já que o resultado é para um par <dataaset, algoritmo>
    #   podemos remover os demais.
    df_ = df[df['dataset'] == dataset].drop(columns=['dataset'])
    df_ = df_[df_['algorithm'] == alg].drop(columns=['algorithm'])

    # DataFrame auxiliar para calcular os grupos
    df__ = df_.copy()
    df__['experiment'] = df__['experiment'].map({f'EXP{i}-{j}': f'{j}'
                                                 for i in range(1, 6)
                                                 for j in ['ALL', '75', '100']})
    # Calcular variância
    df__['std_precision'] = df__['std_precision'] ** 2
    df__['std_recall'] = df__['std_recall'] ** 2
    df__['std_f1_score'] = df__['std_f1_score'] ** 2

    # Agregar os resultados
    agg = df__.groupby('experiment').aggregate('mean')

    # Recalcular std
    df__['std_precision'] = np.sqrt(df__['std_precision'])
    df__['std_recall'] = np.sqrt(df__['std_recall'])
    df__['std_f1_score'] = np.sqrt(df__['std_f1_score'])

    return df_.sort_values(by='experiment'), agg


In [12]:
def cell_data(cell_dict):
    return cell_dict['outputs'][0]['data']['text/plain']


In [13]:
INDICES_RESULTS = {
    'ann': [10, 11],
    'svm': [14, 15],
    'knn': [18, 19],
    'dtree': [22, 23]
}

dataframe = None

In [14]:
base_dir = pathlib.Path('results').absolute()
dfs = []

for p in base_dir.iterdir():
    if not p.is_file():
        # Caso não seja um arquivo, ir para o próximo
        continue
    """
    name = p.name
    if name.startswith('standardized_'):
        ids = name.replace('.ipynb', '').split('_')
        exp = f'exp{ids[2]}'.lower()
        n = ids[3].lower()

        os.rename(str(p), str(p.parent.joinpath(f'standardized_{exp}_{n}.ipynb')))
    """

    ids = p.name.replace('.ipynb', '').split('_')
    dataset = ids[0]
    exp = f'{ids[1]}-{ids[2]}'.upper()

    if ids[0].startswith('min'):
        dataset = f'{ids[0]}_{ids[1]}'
        exp = f'{ids[2]}-{ids[3]}'.upper()

    data = open_result_ipynb(p)

    for alg in INDICES_RESULTS:
        cells = INDICES_RESULTS[alg]

        mean_list = cell_data(data['cells'][cells[0]])
        std_list = cell_data(data['cells'][cells[1]])
        dfs.append(parse_result_dataframe(mean_list,
                                          std_list,
                                          alg,
                                          exp,
                                          dataset))


df = pd.concat(dfs)

In [15]:
df

,dataset,algorithm,experiment,mean_precision,std_precision,mean_recall,std_recall,mean_f1_score,std_f1_score
0,standardized,ann,EXP1-75,0.742632,0.049927,0.724444,0.055134,0.723809,0.053440
0,standardized,svm,EXP1-75,0.732159,0.048747,0.706667,0.046319,0.705974,0.048554
0,standardized,knn,EXP1-75,0.663942,0.070368,0.630000,0.072776,0.626473,0.073021
0,standardized,dtree,EXP1-75,0.481375,0.038089,0.445556,0.039701,0.443465,0.034980
0,standardized,ann,EXP3-100,0.769035,0.047619,0.744444,0.059720,0.744761,0.058463
...,...,...,...,...,...,...,...,...,...
0,raw,dtree,EXP2-100,0.422849,0.066612,0.403333,0.067087,0.400734,0.063700
0,standardized,ann,EXP1-ALL,0.727357,0.043790,0.708889,0.041837,0.705178,0.043721
0,standardized,svm,EXP1-ALL,0.739651,0.053644,0.716667,0.063559,0.714783,0.063346
0,standardized,knn,EXP1-ALL,0.687903,0.043830,0.645556,0.036458,0.641992,0.039810


# Resultados Baseline

In [19]:
df_, agg = results_for(df, 'raw', 'ann')

In [20]:
df_

,experiment,mean_precision,std_precision,mean_recall,std_recall,mean_f1_score,std_f1_score
0,EXP1-100,0.407059,0.173522,0.312222,0.143487,0.274710,0.151235
0,EXP1-75,0.367515,0.138631,0.368889,0.102346,0.318414,0.109191
0,EXP1-ALL,0.446538,0.074424,0.367778,0.056765,0.337627,0.052175
0,EXP2-100,0.435202,0.062213,0.382222,0.054735,0.340208,0.056571
0,EXP2-75,0.382889,0.168660,0.310000,0.132658,0.276638,0.138671
0,EXP2-ALL,0.442108,0.139856,0.385556,0.114042,0.350574,0.111341
0,EXP3-100,0.380904,0.167164,0.343333,0.125866,0.306926,0.137216
0,EXP3-75,0.404432,0.119297,0.353333,0.124920,0.322169,0.122713
0,EXP3-ALL,0.455939,0.177857,0.406667,0.140565,0.363540,0.139968
0,EXP4-100,0.423967,0.059706,0.387778,0.081389,0.349298,0.076987


In [21]:
agg

,mean_precision,std_precision,mean_recall,std_recall,mean_f1_score,std_f1_score
experiment,,,,,,
100,0.407739,0.024645,0.351333,0.014158,0.315235,0.016278
75,0.397064,0.019999,0.347333,0.012769,0.311482,0.013713
ALL,0.459335,0.020064,0.382222,0.010085,0.349754,0.010650
